# Work in Progress/Add in Future:

- Scrape data for min. 1 week
- Change cities

In [1]:
import os
import sys
from datetime import datetime as dt
from datetime import date
from datetime import timedelta
import glob
import numpy as np
import pandas as pd

In [2]:
sys.path.insert(0, os.path.abspath('..\modules'))

In [3]:
import pricemate as pm

In [4]:
# scrape data using pricemate module
df_today = pm.multiple_days_df(0,20)


Looking for [chromedriver 79.0.3945.36 win32] driver in cache 
File found in cache by path [C:\Users\Zenbook\.wdm\drivers\chromedriver\79.0.3945.36\win32\chromedriver.exe]


In [30]:
# add scrape_date
df_today['scrape_datetime'] = dt.now().strftime('%Y-%m-%d %H:%M:%S')

In [31]:
# export scrape output to new csv
todaydate = date.today().strftime('%Y%m%d')
df_today.to_csv('..\scrape-csv\gmr-bdg-' + todaydate + '.csv', index=False)

In [32]:
# import and combine past scrape data with today's scrape, if any
df = df_today
files = glob.glob('..\scrape-csv\*.csv')
for file in files:
    scrapedate = file[-12:-4]
    if scrapedate < todaydate:
        pastdf = pd.read_csv(file)
        df.append(pastdf)
        
print('Earliest scrape: ' + str(min(df['scrape_datetime'])))
print('Latest scrape: ' + str(max(df['scrape_datetime'])))

Earliest scrape: 2020-01-08 01:19:46
Latest scrape: 2020-01-08 01:19:46


In [8]:
# duplicate combined dataframe and examine head
df_raw = df


,arrival_date,arrival_time,depart_date,depart_time,price,seats,tclass,title,scrape_datetime
0,2020-01-08,08:46,2020-01-08,05:30,110000,99,ekonomi (Subclass S),argo parahyangan 36,2020-01-08 00:47:36.598803
1,2020-01-08,09:49,2020-01-08,06:15,110000,99,ekonomi (Subclass S),argo parahyangan 38,2020-01-08 00:47:36.598803
2,2020-01-08,11:19,2020-01-08,07:50,110000,99,ekonomi (Subclass C),pangandaran 176,2020-01-08 00:47:36.598803
3,2020-01-08,12:13,2020-01-08,08:55,110000,99,ekonomi (Subclass S),argo parahyangan 40,2020-01-08 00:47:36.598803
4,2020-01-08,14:09,2020-01-08,10:45,110000,99,ekonomi (Subclass S),argo parahyangan 42,2020-01-08 00:47:36.598803


In [9]:
df.describe()

,arrival_date,arrival_time,depart_date,depart_time,price,seats,tclass,title,scrape_datetime
count,2340,2340,2340,2340,2340,2340.0,2340,2340,2340
unique,22,18,21,18,14,12.0,10,28,1
top,2020-01-19,20:29,2020-01-18,17:10,,0.0,eksekutif (Subclass I),mutiara selatan 104,2020-01-08 00:47:36.598803
freq,116,210,116,210,1457,1457.0,492,210,2340
first,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-01-08 00:47:36.598803
last,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-01-08 00:47:36.598803


In [10]:
df.dtypes
# all data types seem to be objects

arrival_date               object
arrival_time               object
depart_date                object
depart_time                object
price                      object
seats                      object
tclass                     object
title                      object
scrape_datetime    datetime64[ns]
dtype: object

## Data Preprocessing
- Add preprocessing here based on diagnostic testing

In [12]:
# concat date and time, then convert to datetime
df['arrival_datetime'] = pd.to_datetime(df['arrival_date'] + ' ' + df['arrival_time'])
df['depart_datetime'] = pd.to_datetime(df['depart_date'] + ' ' + df['depart_time'])

In [13]:
df[['price', 'seats']] = df[['price', 'seats']].apply(pd.to_numeric)

In [14]:
df[['arrival_date', 'depart_date']] = df[['arrival_date', 'depart_date']].apply(pd.to_datetime)
df['arrival_time'] = pd.to_datetime(df['arrival_time'],format= '%H:%M').dt.time
df['depart_time'] = pd.to_datetime(df['depart_time'],format= '%H:%M').dt.time

## Diagnostic assumptions/test suite
- Arrival date >= depart date
- depart date >= scrape date
- 0 <= seats <= 99


In [16]:
assert min(df['arrival_datetime'] - df['depart_datetime']) > timedelta(0), 'arrival before departure exists'
assert min(df['depart_datetime'] - df['scrape_datetime']) > timedelta(0), 'departure before scrape exists'
assert min(df['seats']) >= 0, 'negative seats exist'
assert max(df['seats']) <= 99, '>99 seats exist'
df.head()

,arrival_date,arrival_time,depart_date,depart_time,price,seats,tclass,title,scrape_datetime,arrival_datetime,depart_datetime
0,2020-01-08,08:46:00,2020-01-08,05:30:00,110000.0,99,ekonomi (Subclass S),argo parahyangan 36,2020-01-08 00:47:36.598803,2020-01-08 08:46:00,2020-01-08 05:30:00
1,2020-01-08,09:49:00,2020-01-08,06:15:00,110000.0,99,ekonomi (Subclass S),argo parahyangan 38,2020-01-08 00:47:36.598803,2020-01-08 09:49:00,2020-01-08 06:15:00
2,2020-01-08,11:19:00,2020-01-08,07:50:00,110000.0,99,ekonomi (Subclass C),pangandaran 176,2020-01-08 00:47:36.598803,2020-01-08 11:19:00,2020-01-08 07:50:00
3,2020-01-08,12:13:00,2020-01-08,08:55:00,110000.0,99,ekonomi (Subclass S),argo parahyangan 40,2020-01-08 00:47:36.598803,2020-01-08 12:13:00,2020-01-08 08:55:00
4,2020-01-08,14:09:00,2020-01-08,10:45:00,110000.0,99,ekonomi (Subclass S),argo parahyangan 42,2020-01-08 00:47:36.598803,2020-01-08 14:09:00,2020-01-08 10:45:00


## Exploratory questions & assumptions
1. earliest available ekonomi train for each day of the week
2. which times and days are the least available?
3. When do the least available tickets start becoming unavailable?